In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "anywidget>=0.9.0",
#     "jupyter-scatter>=0.21.0",
#     "llvmlite>=0.44.0",
#     "numpy>=1.26.0",
#     "pandas>=2.0.0",
#     "scanpy>=1.9.0",
#     "scipy>=1.11.0",
#     "ipywidgets>=8.0.0",
#     "matplotlib>=3.7.0",
#     "requests>=2.31.0",
#     "watchfiles>=0.20.0",
# ]
#
# [tool.uv.sources]
# scsketch = { path = ".", editable = true }
# ///

# Welcome to scSketch

This notebook shows the basic scSketch workflow in a Jupyter notebook.

**Quick start**
- No clone: `uvx scsketch demo`
- From this repo: `uvx juv run demo.ipynb`

You need an `AnnData` object with a 2D embedding in `adata.obsm["X_umap"]` and expression in `adata.X`.
This demo covers (1) Directional Analysis (brush) and (2) Differential Expression (freeform lasso).


## Load Data

Load your single-cell data using scanpy and prepare it for visualization.

In [ ]:
# Dataset download (shows a widget progress bar while downloading)

import os.path
import urllib.request

import ipywidgets as ipyw
from IPython.display import display

# Pick a dataset (downloaded automatically; PBMC3k recommended to get started, pbmc12M.h5ad is very big, 4.6 GB!):
data_file = "pbmc3k.h5ad"
data_url = "https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad"

# data_file = "pbmc12M.h5ad"
# data_url = "https://datasets.cellxgene.cziscience.com/c3d1a5e6-780b-4fe9-a39b-1864f927e87b.h5ad"

# This is just a progress bar to track the file download - don't worry about this code.
status = ipyw.HTML("")
bar = ipyw.IntProgress(value=0, min=0, max=100, description="Download", layout=ipyw.Layout(width="100%"))
ui = ipyw.VBox([status])
display(ui)

def _format_bytes(n: int) -> str:
    n = float(n)
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if n < 1024 or unit == "TB":
            return f"{int(n)} {unit}" if unit == "B" else f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

if os.path.exists(data_file):
    size = os.path.getsize(data_file)
    status.value = f"<b>Dataset:</b> Found <code>{data_file}</code> ({_format_bytes(size)})."
else:
    if not data_url:
        status.value = f"<b>Dataset:</b> Missing <code>{data_file}</code>. Set <code>data_url</code> to download."
    else:
        ui.children = [status, bar]
        status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code>…"

        def _hook(blocknum: int, blocksize: int, totalsize: int):
            downloaded = blocknum * blocksize
            if totalsize and totalsize > 0:
                pct = int(min(100, downloaded * 100 / totalsize))
                bar.value = pct
                status.value = (
                    f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {pct}% "
                    f"({_format_bytes(min(downloaded, totalsize))} / {_format_bytes(totalsize)})"
                )
            else:
                status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {_format_bytes(downloaded)}"

        urllib.request.urlretrieve(data_url, data_file, reporthook=_hook)
        bar.value = 100
        size = os.path.getsize(data_file)
        status.value = f"<b>Dataset:</b> Downloaded <code>{data_file}</code> ({_format_bytes(size)})."


In [ ]:
# Read in the AnnData object file using scanpy
import scanpy as sc

adata = sc.read(data_file)
adata

## Launch scSketch

Create and display the scSketch widget.

In [ ]:
from scsketch import ScSketch

# Pick a reasonable default metadata column if present.
preferred = ["louvain", "leiden", "cell_type", "seurat_clusters"]
metadata_cols = [c for c in preferred if c in adata.obs.columns]
if not metadata_cols and len(adata.obs.columns) > 0:
    metadata_cols = [str(adata.obs.columns[0])]

color_by_default = metadata_cols[0] if metadata_cols else "x"

sketch = ScSketch(
    adata=adata,
    metadata_cols=metadata_cols if metadata_cols else None,
    color_by_default=color_by_default,
    height=720,
    background_color="#111111",
)
sketch.show()


## Directional analysis (Brush)

![](docs/assets/dir_analysis_demo.gif)

1. Set **Lasso Type** to **Brush**.
2. Set **Brush Size** to desired size.
3. Brush-select cells along a direction of interest.
4. Click **+** to save the selection.
5. Click **Compute Directional Search**.
6. Click a gene to view expression vs projection along the direction.
7. Use the Reactome pathway table/diagram to interpret hits.

The results table reports Pearson correlation (`R`) and p-value (`p`) for genes associated with the sketched direction.


## Differential expression (Freeform lasso)

![](docs/assets/diff_exp_analysis_demo.gif)

1. Set **Lasso Type** to **Freeform**.
2. Lasso-select a region of cells.
3. Click **Compute DE** to compare selected vs background cells.
4. Click a gene to view the selected vs background expression distribution plot.

The results table reports Welch’s t-statistic (`T`) and p-value (`p`).


## Programmatic access

Directional results for saved selections can be retrieved as a DataFrame:


In [ ]:
# After saving + computing a selection, retrieve its directional results:
sketch.get_genes("Selection 1")

<details>
<summary>Optional: display gene symbols instead of IDs</summary>

scSketch displays genes using `adata.var_names`. If your dataset stores gene symbols in `adata.var["gene_symbols"]`, you can make a visualization-only copy:

```python
adata_view = adata.copy()
if "gene_symbols" in adata_view.var:
    adata_view.var["ensembl_id"] = adata_view.var_names
    adata_view.var_names = adata_view.var["gene_symbols"].astype(str)
    adata_view.var_names_make_unique()

sketch = ScSketch(
    adata=adata_view,
    metadata_cols=metadata_cols if metadata_cols else None,
    color_by_default=color_by_default,
    height=720,
    background_color="#111111",
)
sketch.show()
```

</details>
